<a href="https://colab.research.google.com/github/nalrob/Aprendizaje-Automatico/blob/main/CTRCD_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Importing the Libraries
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, precision_recall_curve
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)


import warnings
warnings.filterwarnings('ignore')

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, roc_auc_score, recall_score,
                             f1_score, precision_score, confusion_matrix)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [1]:
from google.colab import files
uploaded= files.upload()

Saving cardiotox.zip to cardiotox.zip


In [2]:
import zipfile
import io
data=zipfile.ZipFile(io.BytesIO(uploaded['cardiotox.zip']),'r')
data.extractall()

In [29]:
### SOLO historial clinico

import pandas as pd

def import_data(from_file):
    results = []
    titles = []

    try:
        with open(from_file, 'r') as archivo:
            lineas = archivo.readlines()

            # Process the first line to extract titles
            titles = lineas[0].strip().split(';')

            # Process the missing lines
            for linea in lineas[1:]:
                values = linea.strip().split(';')
                result_iteration = process_values(values)
                results.append(result_iteration)

    except FileNotFoundError:
        print(f"Error: El archivo {from_file} no fue encontrado.")
        return None
    except Exception as e:
        print(f"Error al leer el archivo: {str(e)}")
        return None

    # Create a dataframe w results & add titles
    df_var = pd.DataFrame(results, columns=titles[:27])
    return df_var

def process_values(values):
    # Devolver los valores de las 27 primeras columnas
    return values[:27]

file = '/content/BC_cardiotox_clinical_and_functional_variables.csv'
df = import_data(file)
df

,heart_rate,age,weight,height,CTRCD,time,heart_rhythm,LVEF,PWT,LAd,...,smoker,exsmoker,ACprev,antiHER2prev,RTprev,CIprev,ICMprev,ARRprev,VALVprev,cxvalv
0,103,58,58,151,1,619,0,"60,24096386","0,8","3,8",...,,,,,,,,,,
1,94,75,66,150,0,1884,0,"66,27403031","1,093170204","3,463273568",...,0,0,0,0,0,0,0,0,0,0
2,96,53,55,149,0,1830,0,70,"0,661084178","2,894431584",...,,,,,,,,,,
3,58,67,50,"162,5",1,820,0,"56,54077998","0,798487967","3,439739698",...,,,,,,,,,,
4,76,59,70,159,0,2276,0,"67,21281916","0,73427974","2,698600354",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,69,40,58,"158,5",0,392,0,"67,34021706","0,849898211","2,604906554",...,0,0,0,0,0,0,0,0,0,0
527,77,42,102,163,0,538,0,"73,55165001","1,020905154","2,263032997",...,0,0,0,0,0,0,0,0,0,0
528,80,45,63,171,0,2027,0,"74,78422784","0,808679193","3,376373508",...,0,0,0,0,0,0,0,0,0,0
529,91,53,69,150,0,679,0,"60,81284513","1,078942241","2,571179447",...,,,,,,,,,,


In [30]:
df = df.apply(pd.to_numeric, errors='coerce')

In [31]:
df.fillna(0, inplace=True)

In [32]:
data = df[df['AC'] == 1]

In [33]:
data.columns

Index(['heart_rate', 'age', 'weight', 'height', 'CTRCD', 'time',
       'heart_rhythm', 'LVEF', 'PWT', 'LAd', 'LVDd', 'LVSd', 'AC', 'antiHER2',
       'HTA', 'DL', 'DM', 'smoker', 'exsmoker', 'ACprev', 'antiHER2prev',
       'RTprev', 'CIprev', 'ICMprev', 'ARRprev', 'VALVprev', 'cxvalv'],
      dtype='object')

In [34]:
#scale = ['heart_rate', 'age', 'weight', 'height', 'time', 'LVEF', 'PWT', 'LAd', 'LVSd']
#ohe = ['antiHER2', 'HTA', 'DL', 'DM', 'smoker', 'exsmoker', 'ACprev', 'antiHER2prev', 'RTprev', 'VALVprev']

data = data.drop(columns=['AC','LVDd','CIprev','cxvalv','heart_rhythm','ICMprev','ARRprev','VALVprev'])
data

,heart_rate,age,weight,height,CTRCD,time,LVEF,PWT,LAd,LVSd,antiHER2,HTA,DL,DM,smoker,exsmoker,ACprev,antiHER2prev,RTprev
6,74,65,54.0,164.0,1,252,0.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,69,48,55.0,170.0,0,1836,58.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
10,81,54,73.0,160.0,0,2483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
13,85,56,64.0,158.0,0,1627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,66,43,80.0,155.0,0,363,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,97,40,62.0,0.0,0,591,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,82,37,62.0,161.0,0,198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
525,63,57,60.0,166.0,0,966,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
526,69,40,58.0,0.0,0,392,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
X = data.drop(columns=['CTRCD'])
y = data['CTRCD']

In [58]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir el preprocesamiento
scale = ['heart_rate', 'age', 'weight', 'height', 'time', 'LVEF', 'PWT', 'LAd', 'LVSd']
ohe = ['antiHER2', 'HTA', 'DL', 'DM', 'smoker', 'exsmoker', 'ACprev', 'antiHER2prev', 'RTprev']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ohe)
    ])


# Número de grupos
n_groups = 5

# Inicializar el modelo Random Forest
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier(random_state=42))])

# Almacenar métricas promedio y resultados de cada grupo
metrics = {'accuracy': [], 'roc_auc': [], 'recall': [], 'f1': [], 'precision': []}
group_results = []  # Para almacenar resultados de cada grupo

# Crear grupos balanceados y evaluar el modelo
for group_index in range(n_groups):
    # Separar las clases
    X_train_0 = X_train[y_train == 0]  # Muestras de la clase 0 (mayoritaria)
    X_train_1 = X_train[y_train == 1]  # Muestras de la clase 1 (minoritaria)

    # Mantener todas las muestras de la clase 1
    X_train_balanced = X_train_1.copy()
    y_train_balanced = y_train[y_train == 1].copy()

    # Muestrear aleatoriamente con reemplazo 37 muestras de la clase 0
    X_train_0_sampled = X_train_0.sample(n=37, replace=True, random_state=group_index)
    y_train_0_sampled = y_train.loc[X_train_0_sampled.index]

    # Añadir las muestras de la clase 0 al conjunto balanceado
    X_train_balanced = pd.concat([X_train_balanced, X_train_0_sampled])
    y_train_balanced = pd.concat([y_train_balanced, y_train_0_sampled])

    # Entrenar el modelo
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]) if hasattr(model, "predict_proba") else None
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Almacenar métricas
    metrics['accuracy'].append(accuracy)
    metrics['roc_auc'].append(roc_auc if roc_auc is not None else 0)
    metrics['recall'].append(recall)
    metrics['f1'].append(f1)
    metrics['precision'].append(precision)

    # Almacenar resultados de este grupo
    group_results.append({
        'Group': group_index + 1,
        'Accuracy': accuracy,
        'ROC AUC': roc_auc if roc_auc is not None else 0,
        'Recall': recall,
        'F1 Score': f1,
        'Precision': precision
    })

# Calcular y mostrar el promedio de métricas
print("\nResultados para Random Forest:")
print(f"Promedio Accuracy: {np.mean(metrics['accuracy']):.2f}")
print(f"Promedio ROC AUC: {np.mean(metrics['roc_auc']):.2f}")
print(f"Promedio Recall: {np.mean(metrics['recall']):.2f}")
print(f"Promedio F1 Score: {np.mean(metrics['f1']):.2f}")
print(f"Promedio Precision: {np.mean(metrics['precision']):.2f}")

# Mostrar resultados de cada grupo
print("\nResultados de cada grupo:")
for result in group_results:
    print(f"Grupo {result['Group']} - Accuracy: {result['Accuracy']:.2f}, "
          f"ROC AUC: {result['ROC AUC']:.2f}, Recall: {result['Recall']:.2f}, "
          f"F1 Score: {result['F1 Score']:.2f}, Precision: {result['Precision']:.2f}")



Resultados para Random Forest:
Promedio Accuracy: 0.76
Promedio ROC AUC: 0.59
Promedio Recall: 0.23
Promedio F1 Score: 0.20
Promedio Precision: 0.18

Resultados de cada grupo:
Grupo 1 - Accuracy: 0.79, ROC AUC: 0.66, Recall: 0.31, F1 Score: 0.28, Precision: 0.25
Grupo 2 - Accuracy: 0.81, ROC AUC: 0.52, Recall: 0.23, F1 Score: 0.24, Precision: 0.25
Grupo 3 - Accuracy: 0.76, ROC AUC: 0.65, Recall: 0.23, F1 Score: 0.20, Precision: 0.18
Grupo 4 - Accuracy: 0.73, ROC AUC: 0.60, Recall: 0.23, F1 Score: 0.18, Precision: 0.15
Grupo 5 - Accuracy: 0.70, ROC AUC: 0.53, Recall: 0.15, F1 Score: 0.12, Precision: 0.10


In [59]:
# Dividir en conjuntos de entrenamiento y prueba con estratificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Definir el preprocesamiento
scale = ['heart_rate', 'age', 'weight', 'height', 'time', 'LVEF', 'PWT', 'LAd', 'LVSd']
ohe = ['antiHER2', 'HTA', 'DL', 'DM', 'smoker', 'exsmoker', 'ACprev', 'antiHER2prev', 'RTprev']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ohe)
    ]
)

# Inicializar el modelo Naive Bayes
model = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', GaussianNB())])

# Número de grupos
n_groups = 5

# Almacenar métricas promedio y resultados de cada grupo
metrics = {'accuracy': [], 'roc_auc': [], 'recall': [], 'f1': [], 'precision': []}
group_results = []

# Crear grupos balanceados y evaluar el modelo
for group_index in range(n_groups):
    # Separar las clases
    X_train_0 = X_train[y_train == 0]
    X_train_1 = X_train[y_train == 1]

    # Mantener todas las muestras de la clase 1
    X_train_balanced = X_train_1.copy()
    y_train_balanced = y_train[y_train == 1].copy()

    # Muestrear aleatoriamente con reemplazo 37 muestras de la clase 0
    X_train_0_sampled = X_train_0.sample(n=37, replace=True, random_state=group_index)
    y_train_0_sampled = y_train.loc[X_train_0_sampled.index]

    # Añadir las muestras de la clase 0 al conjunto balanceado
    X_train_balanced = pd.concat([X_train_balanced, X_train_0_sampled])
    y_train_balanced = pd.concat([y_train_balanced, y_train_0_sampled])

    # Entrenar el modelo
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]) if hasattr(model, "predict_proba") else None
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Almacenar métricas
    metrics['accuracy'].append(accuracy)
    metrics['roc_auc'].append(roc_auc if roc_auc is not None else 0)
    metrics['recall'].append(recall)
    metrics['f1'].append(f1)
    metrics['precision'].append(precision)

    # Almacenar resultados de este grupo
    group_results.append({
        'Group': group_index + 1,
        'Accuracy': accuracy,
        'ROC AUC': roc_auc if roc_auc is not None else 0,
        'Recall': recall,
        'F1 Score': f1,
        'Precision': precision
    })

# Calcular y mostrar el promedio de métricas
print("\nResultados para Naive Bayes:")
print(f"Promedio Accuracy: {np.mean(metrics['accuracy']):.2f}")
print(f"Promedio ROC AUC: {np.mean(metrics['roc_auc']):.2f}")
print(f"Promedio Recall: {np.mean(metrics['recall']):.2f}")
print(f"Promedio F1 Score: {np.mean(metrics['f1']):.2f}")
print(f"Promedio Precision: {np.mean(metrics['precision']):.2f}")

# Mostrar resultados de cada grupo
print("\nResultados de cada grupo:")
for result in group_results:
    print(f"Grupo {result['Group']} - Accuracy: {result['Accuracy']:.2f}, "
          f"ROC AUC: {result['ROC AUC']:.2f}, Recall: {result['Recall']:.2f}, "
          f"F1 Score: {result['F1 Score']:.2f}, Precision: {result['Precision']:.2f}")



Resultados para Naive Bayes:
Promedio Accuracy: 0.63
Promedio ROC AUC: 0.70
Promedio Recall: 0.60
Promedio F1 Score: 0.29
Promedio Precision: 0.27

Resultados de cada grupo:
Grupo 1 - Accuracy: 0.32, ROC AUC: 0.59, Recall: 0.82, F1 Score: 0.21, Precision: 0.12
Grupo 2 - Accuracy: 0.78, ROC AUC: 0.80, Recall: 0.64, F1 Score: 0.39, Precision: 0.28
Grupo 3 - Accuracy: 0.89, ROC AUC: 0.78, Recall: 0.18, F1 Score: 0.27, Precision: 0.50
Grupo 4 - Accuracy: 0.81, ROC AUC: 0.65, Recall: 0.55, F1 Score: 0.39, Precision: 0.30
Grupo 5 - Accuracy: 0.35, ROC AUC: 0.67, Recall: 0.82, F1 Score: 0.22, Precision: 0.13


In [60]:
# Dividir en conjuntos de entrenamiento y prueba con estratificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Definir el preprocesamiento
scale = ['heart_rate', 'age', 'weight', 'height', 'time', 'LVEF', 'PWT', 'LAd', 'LVSd']
ohe = ['antiHER2', 'HTA', 'DL', 'DM', 'smoker', 'exsmoker', 'ACprev', 'antiHER2prev', 'RTprev']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ohe)
    ]
)

# Inicializar el modelo KNN
model = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())])

# Número de grupos
n_groups = 5

# Almacenar métricas promedio y resultados de cada grupo
metrics = {'accuracy': [], 'roc_auc': [], 'recall': [], 'f1': [], 'precision': []}
group_results = []

# Crear grupos balanceados y evaluar el modelo
for group_index in range(n_groups):
    # Separar las clases
    X_train_0 = X_train[y_train == 0]
    X_train_1 = X_train[y_train == 1]

    # Mantener todas las muestras de la clase 1
    X_train_balanced = X_train_1.copy()
    y_train_balanced = y_train[y_train == 1].copy()

    # Muestrear aleatoriamente con reemplazo 37 muestras de la clase 0
    X_train_0_sampled = X_train_0.sample(n=37, replace=True, random_state=group_index)
    y_train_0_sampled = y_train.loc[X_train_0_sampled.index]

    # Añadir las muestras de la clase 0 al conjunto balanceado
    X_train_balanced = pd.concat([X_train_balanced, X_train_0_sampled])
    y_train_balanced = pd.concat([y_train_balanced, y_train_0_sampled])

    # Entrenar el modelo
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]) if hasattr(model, "predict_proba") else None
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Almacenar métricas
    metrics['accuracy'].append(accuracy)
    metrics['roc_auc'].append(roc_auc if roc_auc is not None else 0)
    metrics['recall'].append(recall)
    metrics['f1'].append(f1)
    metrics['precision'].append(precision)

# Almacenar resultados de este grupo
    group_results.append({
        'Group': group_index + 1,
        'Accuracy': accuracy,
        'ROC AUC': roc_auc if roc_auc is not None else 0,
        'Recall': recall,
        'F1 Score': f1,
        'Precision': precision
    })

# Calcular y mostrar el promedio de métricas
print("\nResultados para KNN:")
print(f"Promedio Accuracy: {np.mean(metrics['accuracy']):.2f}")
print(f"Promedio ROC AUC: {np.mean(metrics['roc_auc']):.2f}")
print(f"Promedio Recall: {np.mean(metrics['recall']):.2f}")
print(f"Promedio F1 Score: {np.mean(metrics['f1']):.2f}")
print(f"Promedio Precision: {np.mean(metrics['precision']):.2f}")

# Mostrar resultados de cada grupo
print("\nResultados de cada grupo:")
for result in group_results:
    print(f"Grupo {result['Group']} - Accuracy: {result['Accuracy']:.2f}, "
          f"ROC AUC: {result['ROC AUC']:.2f}, Recall: {result['Recall']:.2f}, "
          f"F1 Score: {result['F1 Score']:.2f}, Precision: {result['Precision']:.2f}")


Resultados para KNN:
Promedio Accuracy: 0.71
Promedio ROC AUC: 0.63
Promedio Recall: 0.51
Promedio F1 Score: 0.28
Promedio Precision: 0.19

Resultados de cada grupo:
Grupo 1 - Accuracy: 0.65, ROC AUC: 0.60, Recall: 0.55, F1 Score: 0.26, Precision: 0.17
Grupo 2 - Accuracy: 0.68, ROC AUC: 0.52, Recall: 0.36, F1 Score: 0.20, Precision: 0.14
Grupo 3 - Accuracy: 0.70, ROC AUC: 0.66, Recall: 0.55, F1 Score: 0.29, Precision: 0.19
Grupo 4 - Accuracy: 0.76, ROC AUC: 0.72, Recall: 0.64, F1 Score: 0.37, Precision: 0.26
Grupo 5 - Accuracy: 0.76, ROC AUC: 0.66, Recall: 0.45, F1 Score: 0.29, Precision: 0.22


In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, precision_score

# Supongamos que 'data' es tu DataFrame y 'CTRCD' es la variable objetivo
X = data.drop(columns=['CTRCD'])
y = data['CTRCD']

# Dividir en conjuntos de entrenamiento y prueba con estratificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Definir el preprocesamiento
scale = ['heart_rate', 'age', 'weight', 'height', 'time', 'LVEF', 'PWT', 'LAd', 'LVSd']
ohe = ['antiHER2', 'HTA', 'DL', 'DM', 'smoker', 'exsmoker', 'ACprev', 'antiHER2prev', 'RTprev']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ohe)
    ]
)

# Inicializar el modelo KNN
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())])

# Definir parámetros para Grid Search en KNN
param_grid_knn = {
    'classifier__n_neighbors': [3, 5, 7, 9],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__p': [1, 2]  # p=1 para Manhattan, p=2 para Euclídea
}

# Inicializar el Grid Search
grid_search_knn = GridSearchCV(knn_pipeline, param_grid_knn, cv=5, scoring='accuracy')

# Número de grupos
n_groups = 20

# Almacenar métricas promedio y resultados de cada grupo
metrics = {'accuracy': [], 'roc_auc': [], 'recall': [], 'f1': [], 'precision': []}
group_results = []

# Crear grupos balanceados y evaluar el modelo
for group_index in range(n_groups):
    # Separar las clases
    X_train_0 = X_train[y_train == 0]
    X_train_1 = X_train[y_train == 1]

    # Mantener todas las muestras de la clase 1
    X_train_balanced = X_train_1.copy()
    y_train_balanced = y_train[y_train == 1].copy()

    # Muestrear aleatoriamente con reemplazo 37 muestras de la clase 0
    X_train_0_sampled = X_train_0.sample(n=37, replace=True, random_state=group_index)
    y_train_0_sampled = y_train.loc[X_train_0_sampled.index]

    # Añadir las muestras de la clase 0 al conjunto balanceado
    X_train_balanced = pd.concat([X_train_balanced, X_train_0_sampled])
    y_train_balanced = pd.concat([y_train_balanced, y_train_0_sampled])

    # Entrenar el modelo con Grid Search para encontrar los mejores parámetros
    grid_search_knn.fit(X_train_balanced, y_train_balanced)

    # Realizar predicciones con el mejor modelo encontrado
    y_pred = grid_search_knn.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, grid_search_knn.predict_proba(X_test)[:, 1]) if hasattr(grid_search_knn, "predict_proba") else None
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Almacenar métricas
    metrics['accuracy'].append(accuracy)
    metrics['roc_auc'].append(roc_auc if roc_auc is not None else 0)
    metrics['recall'].append(recall)
    metrics['f1'].append(f1)
    metrics['precision'].append(precision)

    # Almacenar resultados de este grupo
    group_results.append({
        'Group': group_index + 1,
        'Accuracy': accuracy,
        'ROC AUC': roc_auc if roc_auc is not None else 0,
        'Recall': recall,
        'F1 Score': f1,
        'Precision': precision
    })

# Calcular y mostrar el promedio de métricas
print("\nResultados para KNN con Grid Search:")
print(f"Promedio Accuracy: {np.mean(metrics['accuracy']):.2f}")
print(f"Promedio ROC AUC: {np.mean(metrics['roc_auc']):.2f}")
print(f"Promedio Recall: {np.mean(metrics['recall']):.2f}")
print(f"Promedio F1 Score: {np.mean(metrics['f1']):.2f}")
print(f"Promedio Precision: {np.mean(metrics['precision']):.2f}")

# Mostrar resultados de cada grupo
print("\nResultados de cada grupo:")
for result in group_results:
    print(f"Grupo {result['Group']} - Accuracy: {result['Accuracy']:.2f}, "
          f"ROC AUC: {result['ROC AUC']:.2f}, Recall: {result['Recall']:.2f}, "
          f"F1 Score: {result['F1 Score']:.2f}, Precision: {result['Precision']:.2f}")

# Mostrar los mejores parámetros encontrados por Grid Search
print(f"\nMejores parámetros encontrados por Grid Search: {grid_search_knn.best_params_}")



Resultados para KNN con Grid Search:
Promedio Accuracy: 0.70
Promedio ROC AUC: 0.62
Promedio Recall: 0.41
Promedio F1 Score: 0.23
Promedio Precision: 0.16

Resultados de cada grupo:
Grupo 1 - Accuracy: 0.65, ROC AUC: 0.59, Recall: 0.36, F1 Score: 0.19, Precision: 0.12
Grupo 2 - Accuracy: 0.71, ROC AUC: 0.52, Recall: 0.27, F1 Score: 0.17, Precision: 0.12
Grupo 3 - Accuracy: 0.70, ROC AUC: 0.64, Recall: 0.36, F1 Score: 0.21, Precision: 0.15
Grupo 4 - Accuracy: 0.77, ROC AUC: 0.74, Recall: 0.55, F1 Score: 0.34, Precision: 0.25
Grupo 5 - Accuracy: 0.71, ROC AUC: 0.71, Recall: 0.55, F1 Score: 0.29, Precision: 0.20
Grupo 6 - Accuracy: 0.78, ROC AUC: 0.69, Recall: 0.27, F1 Score: 0.21, Precision: 0.18
Grupo 7 - Accuracy: 0.69, ROC AUC: 0.64, Recall: 0.45, F1 Score: 0.24, Precision: 0.17
Grupo 8 - Accuracy: 0.69, ROC AUC: 0.71, Recall: 0.45, F1 Score: 0.24, Precision: 0.17
Grupo 9 - Accuracy: 0.69, ROC AUC: 0.54, Recall: 0.45, F1 Score: 0.24, Precision: 0.17
Grupo 10 - Accuracy: 0.73, ROC AUC